# Lex Email Demo

#### This notebook is set up to run on a SageMaker Notebook Instance. 
It requires the IAM Role attached to your SageMaker Notebook Instance to have Lex Permissions attached.

Ensure you understand the pricing for Lex and SageMaker
- https://aws.amazon.com/lex/pricing/
- https://aws.amazon.com/sagemaker/pricing/

#### This notebook requires you to have already set up your Lex Bot
- Navigate to the Lex Console at https://console.aws.amazon.com/lex
- Choose Actions -> Import
- Upload coffee_and_cover_bot_Export.zip included in this repo
- Select the Bot and click navigate to the Editor page
- Select "Build"
- Once built choose "Publish" and choose an alias of "test" (you can select a different name, but be sure to update the bot_alias used in this notebook)

#### Pull in needed libraries

Python SDK for Lex Runtime at https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lex-runtime.html

Note that the SDKs are all available in multiple programming languages https://aws.amazon.com/tools/

In [1]:
import json
import datetime
import boto3
lex = boto3.client('lex-runtime')

#### Set up with some Dummy Emails
In real life these would be ingested from a service or a storage location

In [2]:
email_messages = [
    """
    Hey, want me to grab you a drink on my way in this morning?
    """,
    """
    I'm going to be late this morning, can you take the 10 o'clock?
    """,
    """
    I'm going to the coffee shop, want anything?
    """,
    """
    Can you cover the meeting tomorrow? I have an appointment.
    """
    ]

negative = 2
positive = 1

#### Call Lex

** If you have any issues running the below, ensure you have Set Up, Built and Published your Bot correctly.

In [3]:
bot_name = 'coffee_and_cover_bot'
bot_alias = 'test' # you set this when you Publish the bot
user_id = '123' # you will need to vary this if running at high scale, esp if you expect to hit a lot of missed intents

In [4]:
email_intent = lex.post_text(
    botName=bot_name,
    botAlias=bot_alias,
    userId=user_id, 
    inputText=email_messages[0]
)
print(json.dumps(email_intent, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "e7401648-209f-4b72-a7d9-e382f1edc608",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "e7401648-209f-4b72-a7d9-e382f1edc608",
      "date": "Sat, 25 Jul 2020 17:31:02 GMT",
      "content-type": "application/json",
      "content-length": "273"
    },
    "RetryAttempts": 0
  },
  "intentName": "GetCoffee",
  "slots": {},
  "dialogState": "ReadyForFulfillment",
  "sessionId": "2020-07-25T17:25:12.367Z-hYWAlmJf"
}


#### Loop through emails to determine Intent for each and take appropriate action

In [5]:
for email in email_messages:
    
    print(email)
    
    email_intent = lex.post_text(
        botName=bot_name,
        botAlias=bot_alias,
        userId=user_id, 
        inputText=email
    )
    
    print(email_intent['intentName'])
    
    if email_intent['intentName'] == 'GetCoffee':
        # Set up an auto reply - we don't want to miss coffee!
        print('Auto Reply: YESSS!!')
    
    elif email_intent['intentName'] == 'CoverForMe':
        # Add to Insights gained from Comprehend
        print('Flag as Important')


    print('')
    print('-------')


    Hey, want me to grab you a drink on my way in this morning?
    
GetCoffee
Auto Reply: YESSS!!

-------

    I'm going to be late this morning, can you take the 10 o'clock?
    
CoverForMe
Flag as Important

-------

    I'm going to the coffee shop, want anything?
    
GetCoffee
Auto Reply: YESSS!!

-------

    Can you cover the meeting tomorrow? I have an appointment.
    
CoverForMe
Flag as Important

-------


#### Getting a little fancy - Customize Coffee Order based on overall sentiment of emails in mailbox 

In [6]:
for email in email_messages:
    
    print(email)
    
    email_intent = lex.post_text(
        botName=bot_name,
        botAlias=bot_alias,
        userId=user_id, 
        inputText=email
    )
    
    print(email_intent['intentName'])
    
    if email_intent['intentName'] == 'GetCoffee':
        # Set up an auto reply - we don't want to miss coffee!
        # Get sentiment from insights from comprehend (mocked here)
        if positive > negative:
            # Chill morning
            print('Reply: Yes, I would love a Green Tea!')
        else:
            # Need that extra boost!
            print('Reply: YESSS!! An Americano with extra extra shots please!')
    
    elif email_intent['intentName'] == 'CoverForMe':
        # Add to Insights gained from Comprehend
        print('Flag as Important')


    print('')
    print('-------')


    Hey, want me to grab you a drink on my way in this morning?
    
GetCoffee
Reply: YESSS!! An Americano with extra extra shots please!

-------

    I'm going to be late this morning, can you take the 10 o'clock?
    
CoverForMe
Flag as Important

-------

    I'm going to the coffee shop, want anything?
    
GetCoffee
Reply: YESSS!! An Americano with extra extra shots please!

-------

    Can you cover the meeting tomorrow? I have an appointment.
    
CoverForMe
Flag as Important

-------


#### Getting even fancier with Dates

In [7]:
for email in email_messages:
    
    print(email)
    
    email_intent = lex.post_text(
        botName=bot_name,
        botAlias=bot_alias,
        userId=user_id, 
        inputText=email
    )
    
    print(email_intent['intentName'])
    
    if email_intent['intentName'] == 'GetCoffee':
        # Set up an auto reply - we don't want to miss coffee!
        # Get overall sentiment for all emails we have saved from earlier (mocked here)
        if positive > negative:
            # Chill morning
            print('Reply: Yes, I would love a Green Tea!')
        else:
            # Need that extra boost!
            print('Reply: YESSS!! An Americano with extra extra shots please!')
    
    elif email_intent['intentName'] == 'CoverForMe':
        # We can check the date captured in the Lex Slot
        cover_date_string = email_intent['slots']['date']
        
        if cover_date_string != None:
            # Check if the date is today
            cover_date = datetime.datetime.strptime(cover_date_string, "%Y-%m-%d")
            today = datetime.datetime.now()
            
            if cover_date.date() == today.date():
                # Add to Insights gained from Comprehend - make sure we see it right away
                print('Flag as Important')
                
            else:
                print('//it can wait')
        else:
            print('//it can wait')


    print('')
    print('-------')


    Hey, want me to grab you a drink on my way in this morning?
    
GetCoffee
Reply: YESSS!! An Americano with extra extra shots please!

-------

    I'm going to be late this morning, can you take the 10 o'clock?
    
CoverForMe
Flag as Important

-------

    I'm going to the coffee shop, want anything?
    
GetCoffee
Reply: YESSS!! An Americano with extra extra shots please!

-------

    Can you cover the meeting tomorrow? I have an appointment.
    
CoverForMe
//it can wait

-------


#### Keep Exploring!

|￣￣￣￣￣￣|  
|&nbsp;&nbsp;&nbsp;&nbsp;WHAT'S &nbsp;&nbsp;&nbsp;|  
|&nbsp;&nbsp;&nbsp;&nbsp;NEXT? &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|  
|＿＿＿＿＿＿|  
(\\__/) ||  
(•ㅅ•) ||  
/ 　 づ  


This is just a few examples - check out the Lex Docs and try out some of the other features! 
https://docs.aws.amazon.com/lex/latest/dg/what-is.html

Keep having fun, keep building... :)
@virtualgill